In [1]:
source("~/sc-online/getData.R")
source("~/sc-online/utils.R")
source("~/sc-online/plot.R")
library(qs)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [2]:
# load NURR + DAPI
# Count Libs, Participants

nurr_seurat_clean = qread("/mnt/accessory/seq_data/calico/seurat_nurr_merged_initial_harmonized_20240130.qs")
dapi_seurat_clean = readRDS("/mnt/accessory/seq_data/calico/seurat_dapi_merged_initial_harmonized_20240119.rds")

In [3]:
# add participant ID
manifest = read.table("~/Calico_Macosko_PD_GSA_Terra_Manifest_20240220.tsv", sep='\t',header=T,skip=0)
sort(colnames(manifest))

manifest$donor_id = sapply(manifest$entity.sample_id, function(x) gsub("_1$", "", x))
manifest$sex = manifest$Sex
manifest$age = manifest$Age 
manifest$participant_id = ifelse(
    manifest$CORRECT.donor == '', 
    manifest$collaborator_participant_id,
    manifest$CORRECT.donor
)
manifest$case_control = ifelse(manifest$Disease == 'Control', 'ctr', 'pd')
manifest = manifest[, c('donor_id', 'participant_id', 'case_control', 'sex', 'age')]

donor_participant_map = setNames(manifest$participant_id, manifest$donor_id)

donor_case_control_map = setNames(manifest$case_control, manifest$donor_id)

nurr_seurat_clean$case_control = donor_case_control_map[nurr_seurat_clean$donor_id]
dapi_seurat_clean$case_control = donor_case_control_map[dapi_seurat_clean$donor_id]

nurr_seurat_clean$participant_id = donor_participant_map[nurr_seurat_clean$donor_id]
dapi_seurat_clean$participant_id = donor_participant_map[dapi_seurat_clean$donor_id]

[1] "Age"                         "autocall_call_rate"         
 [3] "autocall_gender"             "autocall_version"           
 [5] "call_date"                   "call_rate"                  
 [7] "called_infinium_snps"        "chip"                       
 [9] "chip_well_barcode"           "collaborator_participant_id"
[11] "collaborator_sample_id"      "CORRECT.donor"              
[13] "Disease"                     "entity.sample_id"           
[15] "fp_gender"                   "fp_lod_score"               
[17] "gender_concordance"          "green_idat"                 
[19] "gtc"                         "hap_map_concordance"        
[21] "het_rate"                    "last_cluster_file"          
[23] "participant"                 "pdo"                        
[25] "red_idat"                    "release_date"               
[27] "reported_gender"             "reported_gender.1"          
[29] "research_project_id"         "scan_date"                  
[31] "Sex"                         "stock_sample"               
[33] "vcf"                         "vcf_index"                  
[35] "version"                     "zcall_version"

In [4]:
donor_participant_map

206954930010_R05C02 206954930010_R09C01 206954930011_R11C02 206954930060_R11C02 
           "PDC091"            "PDC164"            "PDC165"            "PDC110" 
206954930093_R05C02 207762950055_R07C02 207762950055_R11C01 207762950086_R02C01 
           "PDC078"            "PDC078"          "2019-102"            "PD0878" 
207762950108_R10C02 207762960003_R12C02 206954930010_R01C01 206954930010_R01C02 
           "PD0985"            "PD0784"            "PDC090"            "PD0717" 
206954930010_R02C01 206954930010_R03C01 206954930010_R03C02 206954930010_R04C01 
           "PDC139"            "PD0730"            "PD0940"            "PDC078" 
206954930010_R05C01 206954930010_R06C01 206954930010_R07C01 206954930010_R08C01 
           "PD0870"            "PDC092"            "PD0968"            "PD0793" 
206954930010_R10C01 206954930010_R11C01 206954930011_R01C02 206954930011_R02C01 
           "PD0875"         "Uncertain"            "PDC131"            "PD0954" 
206954930011_R02C02 206954930011_R03C01 206954930011_R03C02 206954930011_R04C01 
           "PD0847"            "PDC123"            "PD0911"            "PD0976" 
206954930011_R04C02 206954930011_R05C01 206954930011_R05C02 206954930011_R06C02 
           "PD0980"            "PD0833"            "PD0959"            "PD0985" 
206954930011_R07C01 206954930011_R07C02 206954930011_R08C01 206954930011_R08C02 
           "PD0944"            "PD0775"            "PDC126"            "PD0918" 
206954930011_R09C01 206954930011_R09C02 206954930011_R10C01 206954930011_R10C02 
           "PD0981"            "PDC089"            "PD0825"            "PD0935" 
206954930011_R11C01 206954930011_R12C01 206954930011_R12C02 206954930060_R02C01 
       "Uncertain "            "PD0957"            "PD0897"            "PD0876" 
206954930060_R02C02 206954930060_R03C01 206954930060_R03C02 206954930060_R04C01 
           "PD0800"            "PDC105"            "PD0792"            "PD0834" 
206954930060_R04C02 206954930060_R05C01 206954930060_R05C02 206954930060_R07C01 
           "PD0905"            "PD0765"            "PD0926"            "PD0903" 
206954930060_R07C02 206954930060_R08C01 206954930060_R08C02 206954930060_R10C01 
           "PD0902"            "PDC107"            "PD0801"            "PD0746" 
206954930060_R10C02 206954930060_R11C01 206954930060_R12C01 206954930060_R12C02 
           "PD0839"            "PD0784"            "PD0904"            "PD0936" 
206954930093_R01C02 206954930093_R02C01 206954930093_R02C02 206954930093_R03C01 
           "PDC093"            "PD0970"            "PD0814"            "PDC111" 
206954930093_R03C02 206954930093_R04C01 206954930093_R04C02 206954930093_R05C01 
           "PD0878"            "PD0948"            "PD0938"            "PD0808" 
206954930093_R06C02 206954930093_R07C01 206954930093_R07C02 206954930093_R08C01 
           "PDC112"            "PD0906"            "PD0736"            "PDC114" 
206954930093_R08C02 206954930093_R09C01 206954930093_R10C01 206954930093_R10C02 
           "PD0899"            "PDC088"            "PD0809"            "PDC128" 
206954930093_R11C01 206954930093_R11C02 206954930093_R12C01 207758480019_R01C01 
           "PD0816"            "PDC121"            "PD0916"          "2019-114" 
207762950055_R01C01 207762950055_R02C01 207762950055_R03C02 207762950055_R04C01 
         "2007-075"            "PD0903"          "2019-114"          "2008-028" 
207762950055_R05C02 207762950055_R06C01 207762950055_R08C02 207762950055_R10C02 
           "PD0809"          "1995-078"          "2018-130"            "PD0899" 
207762950055_R12C02 207762950086_R01C02 207762950086_R03C02 207762950086_R04C01 
           "PDC089"          "2019-129"          "2018-130"            "PD0876" 
207762950086_R05C02 207762950086_R06C02 207762950086_R07C02 207762950086_R08C02 
           "PD0968"          "2008-052"          "2009-016"          "2019-071" 
207762950086_R10C02 207762950086_R11C01 207762950086_R12C02 207762950108_R01C02 
           "PD0825"          "2

In [20]:

length(union(unique(dapi_seurat_clean$participant_id), unique(nurr_seurat_clean$participant_id)))

[1] 51

In [19]:
unique(dapi_seurat_clean$participant_id)

[1] "PDC139"     "2008-052"   "2009-016"   "PDC165"     "2002-086"  
 [6] "PDC114"     "PD0793"     "PD0903"     "PD0968"     "Uncertain "
[11] "PD0940"     "PDC110"     "2017-037"   "2008-028"   "2019-114"  
[16] "PDC092"     "2018-130"   "PD0809"     "PDC164"     "PD0847"    
[21] "PDC088"     "2019-017"   "PD0957"     "PD0954"     "PD0899"    
[26] "PD0730"     "2019-071"   "PDC111"     "PDC093"     "PD0870"    
[31] "PD0878"     "PD0935"     "PD0905"     "PD0839"     "PD0897"    
[36] "PD0902"     "PD0736"     "PDC078"     "PD0985"     "PD0938"    
[41] "PD0825"     "PD0926"     "PD0784"     "PDC089"     "PDC094"    
[46] "PD0906"     "PDC091"     "PDC105"

In [22]:
dapi_table = table(dapi_seurat_clean$participant_id, dapi_seurat_clean$case_control)
nurr_table = table(nurr_seurat_clean$participant_id, nurr_seurat_clean$case_control)

In [27]:
sum(dapi_table[, 'ctr'] > 0)
sum(dapi_table[, 'pd'] > 0)
sum(nurr_table[, 'ctr'] > 0)
sum(nurr_table[, 'pd'] > 0)

[1] 18

[1] 30

[1] 18

[1] 26

In [28]:
dim(dapi_seurat_clean)
dim(nurr_seurat_clean)

[1] 31108 64882

[1] 31558 39262

In [6]:
(64882 + 39262) / 36

[1] 2892.889